## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [27]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
from zipfile import ZipFile

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

In [30]:
!wget -P /content/gdrive/My\ Drive https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/3yG0-M4M8fypeQ

--2022-02-09 21:11:09--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/3yG0-M4M8fypeQ
Resolving getfile.dokpub.com (getfile.dokpub.com)... 78.46.92.107
Connecting to getfile.dokpub.com (getfile.dokpub.com)|78.46.92.107|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader.disk.yandex.ru/disk/f21305e83970878d6875050927dd15947ceeae2ae6b2762aa9921c8e564fa9e5/620465f8/HdZOcL9j2UCaFZl90Y8xfcTnpNaEkOFBvBMcb1tjQV6ciMfYRf6DCCWmZlYoeRwMntukoppeKNR68PfXdYslpA%3D%3D?uid=0&filename=cc.ru.300.vec.zip&disposition=attachment&hash=Tf4kWKzkDPdN2THngpCb3pCAfsAgLUptvmXFCRFSUxY%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=78404243&fsize=397254664&hid=67d07e7a6953ad340b81e01c02abd45d&media_type=compressed&tknv=v2 [following]
--2022-02-09 21:11:10--  https://downloader.disk.yandex.ru/disk/f21305e83970878d6875050927dd15947ceeae2ae6b2762aa9921c8e564fa9e5/620465f8/HdZOcL9j2UCaFZl90Y8xfcTnpNaEkOFBvBMcb1tjQV6ciMfYRf6DCCWmZlYoeRwMn

In [31]:
!wget -P /content/gdrive/My\ Drive https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/9CAeNsJiInoyUA

--2022-02-09 21:16:16--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/9CAeNsJiInoyUA
Resolving getfile.dokpub.com (getfile.dokpub.com)... 78.46.92.107
Connecting to getfile.dokpub.com (getfile.dokpub.com)|78.46.92.107|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader.disk.yandex.ru/disk/a87cc6fe1c63769c65d623b96575f8f0b4f3785dfe4bc057c25ac614ce1c70c3/62046749/HdZOcL9j2UCaFZl90Y8xfcevwXkXuXvaIYjUX06MZoTXMyIOD6p5K-p8gA1R1iDaMnWY-AL8LMuEB1uPSIvHDw%3D%3D?uid=0&filename=cc.uk.300.vec.zip&disposition=attachment&hash=0MriIZAL2DprNGr89hL7a6bICrn4LuFQAJY%2BK4qSU1A%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=78404243&fsize=386324207&hid=d843208373e1ea4be77ac5cfc29ca557&media_type=compressed&tknv=v2 [following]
--2022-02-09 21:16:17--  https://downloader.disk.yandex.ru/disk/a87cc6fe1c63769c65d623b96575f8f0b4f3785dfe4bc057c25ac614ce1c70c3/62046749/HdZOcL9j2UCaFZl90Y8xfcevwXkXuXvaIYjUX06MZoTXMyIOD6p5K-p8gA1R1iD

In [32]:
path = "/content/gdrive/My Drive/cc.uk.300.vec.zip"
tar = ZipFile(path, "r")
tar.extractall(r"/content/gdrive/My Drive/cc.uk.300.vec")
tar.close()

In [33]:
path = "/content/gdrive/My Drive/cc.ru.300.vec.zip"
tar = ZipFile(path, "r")
tar.extractall(r"/content/gdrive/My Drive/cc.ru.300.vec")
tar.close()

Load embeddings for ukrainian and russian.

In [35]:
uk_emb = KeyedVectors.load_word2vec_format("/content/gdrive/My Drive/cc.uk.300.vec/cc.uk.300.vec")

In [36]:
ru_emb = KeyedVectors.load_word2vec_format("/content/gdrive/My Drive/cc.ru.300.vec/cc.ru.300.vec")

In [39]:
ru_emb.most_similar([ru_emb["август"]], topn=5)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999)]

In [38]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [40]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796566247940063)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [41]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [42]:
!wget -P /content/gdrive/My\ Drive https://raw.githubusercontent.com/yandexdataschool/nlp_course/2021/week01_embeddings/ukr_rus.test.txt

--2022-02-09 21:28:03--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/2021/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Saving to: ‘/content/gdrive/My Drive/ukr_rus.test.txt’

ukr_rus.test.txt    100%[===================>]  11.90K  --.-KB/s    in 0s      

2022-02-09 21:28:03 (59.0 MB/s) - ‘/content/gdrive/My Drive/ukr_rus.test.txt’ saved [12188/12188]



In [43]:
!wget -P /content/gdrive/My\ Drive https://raw.githubusercontent.com/yandexdataschool/nlp_course/2021/week01_embeddings/ukr_rus.train.txt

--2022-02-09 21:28:43--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/2021/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Saving to: ‘/content/gdrive/My Drive/ukr_rus.train.txt’

ukr_rus.train.txt   100%[===================>]  57.96K  --.-KB/s    in 0.01s   

2022-02-09 21:28:43 (5.27 MB/s) - ‘/content/gdrive/My Drive/ukr_rus.train.txt’ saved [59351/59351]



In [44]:
uk_ru_train, X_train, Y_train = load_word_pairs("/content/gdrive/My Drive/ukr_rus.train.txt")

In [45]:
uk_ru_test, X_test, Y_test = load_word_pairs("/content/gdrive/My Drive/ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [47]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression()

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [48]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541592359542847),
 ('июнь', 0.8411962985992432),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.8359213471412659),
 ('февраль', 0.8328747749328613),
 ('октябрь', 0.8311804533004761),
 ('ноябрь', 0.8278145790100098),
 ('июль', 0.8236349821090698),
 ('август', 0.8120611906051636),
 ('декабрь', 0.8037999272346497)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [74]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        emb = ru_emb.most_similar([mapped_vectors[i]])[:topn]
        for words in emb:
          if words[0] == ru:
              num_matches += 1

    precision_val = num_matches / len(pairs)
    return precision_val


In [75]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [76]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [81]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.811

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [82]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE
    u, Rs, vh = np.linalg.svd(X_train.T @ Y_train, full_matrices=False)
    return u @ vh

In [83]:
W = learn_transform(X_train, Y_train)

In [84]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025654554367065),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001736402511597),
 ('ноябрь', 0.7934483885765076),
 ('февраль', 0.7914121150970459),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [85]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [86]:
!wget -P /content/gdrive/My\ Drive https://raw.githubusercontent.com/yandexdataschool/nlp_course/2021/week01_embeddings/fairy_tale.txt

--2022-02-09 22:32:20--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/2021/week01_embeddings/fairy_tale.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9191 (9.0K) [text/plain]
Saving to: ‘/content/gdrive/My Drive/fairy_tale.txt’

fairy_tale.txt      100%[===================>]   8.98K  --.-KB/s    in 0s      

2022-02-09 22:32:20 (93.2 MB/s) - ‘/content/gdrive/My Drive/fairy_tale.txt’ saved [9191/9191]



In [87]:
with open("/content/gdrive/My Drive/fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [142]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE HERE
    text = sentence.split(' ')
    list_senc = []
    for word in text:
      try:
        ru = ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0]
        list_senc.append(ru)
      except KeyError:
        list_senc.append(word)
    return ' '.join(list_senc)

In [143]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [144]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала хатку, и и живе. а оно приходят холоди. из лисичка замерзла и и побежала во село огня добувать, чтобы витопити. прибегает к одной бабы и и каже:

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые були, бабусю! со неділею... позичте мне огню, мной тебе одслужу.

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — добре, — каже, — лисичко – сестричко. садись погрійся трохи, пока мной пирожки повибираю со печі!

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала смі

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -